# Imports

In [45]:
import pandas as pd
import re
from collections import Counter

In [46]:
pip install emoji

# Pre-Processing

## Read in the data

This data was cleaned in another file and not here just for ease and to convenience but the file used to clean this data set is also attached

In [47]:
df = pd.read_csv('/content/dev_cleaned3.csv')

## Splitting the data into training and testing sets

In [48]:
from sklearn.model_selection import train_test_split

df.dropna(subset=['emoji'], inplace=True)

texts = df['text'].values
labels = df['emoji'].apply(str).values
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


# Model 2: Baseline: No sentiment and No LSTM

A lot of steps for each model are the same or repeated due to the long run time of the models, for example each model was created and ran on different days and thus imports may repeat or parts of the code may repeat because we did not run the entire file at once but instead ran each model at different times

In [56]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np



# filter out NaN values
valid_indices = [i for i, label in enumerate(y_train) if pd.notna(label)]
filtered_texts = [X_train[i] for i in valid_indices]
filtered_labels = [y_train[i] for i in valid_indices]

# Tokenize texts
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(filtered_texts)
sequences = tokenizer.texts_to_sequences(filtered_texts)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=10)

# Encode labels
label_list = np.unique(filtered_labels)
label_encoder = LabelEncoder()
label_encoder.fit(label_list)
encoded_labels = label_encoder.transform(filtered_labels)
y = tf.keras.utils.to_categorical(encoded_labels, num_classes=len(label_list))


# First baseline model- no LSTM layers

model2 = Sequential()
model2.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8, input_length=10))
model2.add(Flatten())
model2.add(Dense(len(label_list), activation='softmax'))



model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()


model2.fit(X, y, epochs=10)



Epoch 1/10
7552/7552 [==============================] - 37s 5ms/step - loss: 2.8606 - accuracy: 0.2935
Epoch 2/10
7552/7552 [==============================] - 26s 3ms/step - loss: 2.5419 - accuracy: 0.3512
Epoch 3/10
7552/7552 [==============================] - 25s 3ms/step - loss: 2.3916 - accuracy: 0.3870
Epoch 4/10
7552/7552 [==============================] - 27s 4ms/step - loss: 2.2814 - accuracy: 0.4139
Epoch 5/10
7552/7552 [==============================] - 25s 3ms/step - loss: 2.1936 - accuracy: 0.4365
Epoch 6/10
7552/7552 [==============================] - 25s 3ms/step - loss: 2.1209 - accuracy: 0.4548
Epoch 7/10
7552/7552 [==============================] - 25s 3ms/step - loss: 2.0599 - accuracy: 0.4712
Epoch 8/10
7552/7552 [==============================] - 25s 3ms/step - loss: 2.0079 - accuracy: 0.4852
Epoch 9/10
7552/7552 [==============================] - 24s 3ms/step - loss: 1.9629 - accuracy: 0.4973
Epoch 10/10
1888/1888 [==============================] - 4s 2ms/step
[':f

## Testing Model 2

In [57]:

# Process the testing data like the training data
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=10)
encoded_labels_test = label_encoder.transform(y_test)
y_test_categorical = tf.keras.utils.to_categorical(encoded_labels_test, num_classes=len(label_list))

# Predict classes and find true classes for accuracy measures
predictions2 = model2.predict(X_test_padded)
predicted_classes = np.argmax(predictions2, axis=1)
true_classes = np.argmax(y_test_categorical, axis=1)



1888/1888 [==============================] - 3s 1ms/step


Originally, we were following the standford paper that uses accuracy, precision, recall, and F1 score to assess their models so we followed these steps as well but only included accuracy in our report as we were not interested in precision, recall, or F1 score.

In [58]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(true_classes, predicted_classes, average='weighted')
accuracy = accuracy_score(true_classes, predicted_classes)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.3613
Precision: 0.3333
Recall: 0.3613
F1 Score: 0.3225


The follow code boxes take random sentances for the NN to predict emojis on for our peers to evalute in the testing phase

In [80]:
# New test sentance
random_sentence = "yay"

# Tokenize and pad the sentence
sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

# Predict the label
prediction = model2.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

# Decode the predicted label
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)


1/1 [==============================] - 0s 17ms/step
Predicted label for 'yay': [':face_with_tears_of_joy:']


In [60]:
random_sentence = "I am so happy"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model2.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 18ms/step
Predicted label for 'I am so happy': [':loudly_crying_face:']


In [ ]:
random_sentence = "Bro I am so mad"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model2.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 17ms/step
Predicted label for 'Bro I am so mad': [':face_with_tears_of_joy:']


In [ ]:
random_sentence = "I love you"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model2.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 17ms/step
Predicted label for 'I love you': [':red_heart:']


In [ ]:
random_sentence = "LMAO"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model2.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 19ms/step
Predicted label for 'LMAO': [':face_with_tears_of_joy:']


# Model 3: LSTM with no sentiment

Reference for bidirectional LSTMs: https://keras.io/api/layers/recurrent_layers/bidirectional/

In [ ]:
max_seq_length = 10
vocab_size = len(label_list)

model3 = Sequential()
model3.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_seq_length))
model3.add(Bidirectional(LSTM(64, return_sequences=True)))
model3.add(Dropout(0.5))
model3.add(LSTM(64))
model3.add(Dense(len(label_list), activation='softmax'))

model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

model3.fit(X, y, epochs=10)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 10, 128)           6144      
                                                                 
 bidirectional_1 (Bidirecti  (None, 10, 128)           98816     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 48)                3120      
                                                                 
Total params: 157488 (615.19 KB)
Trainable params: 157488 (615.19 KB)
Non-trainable params: 0 (0.00 Byte)
______________

## Testing Model 3

In [ ]:

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=10)
encoded_labels_test = label_encoder.transform(y_test)
y_test_categorical = tf.keras.utils.to_categorical(encoded_labels_test, num_classes=len(label_list))


predictions2 = model3.predict(X_test_padded)
predicted_classes = np.argmax(predictions2, axis=1)
true_classes = np.argmax(y_test_categorical, axis=1)



In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(true_classes, predicted_classes, average='weighted')
accuracy = accuracy_score(true_classes, predicted_classes)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.3234
Precision: 0.4291
Recall: 0.3234
F1 Score: 0.2428


In [ ]:

random_sentence = "yay"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model3.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)


1/1 [==============================] - 0s 114ms/step
Predicted label for 'yay': [':clapping_hands:']


In [ ]:
random_sentence = "I am so happy"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model3.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)


1/1 [==============================] - 0s 71ms/step
Predicted label for 'I am so happy': [':loudly_crying_face:']


In [ ]:
random_sentence = "Bro I am so mad"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model3.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)




1/1 [==============================] - 0s 28ms/step
Predicted label for 'Bro I am so mad': [':loudly_crying_face:']


In [ ]:
random_sentence = "I love you"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model3.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)



1/1 [==============================] - 0s 19ms/step
Predicted label for 'I love you': [':two_hearts:']


In [ ]:
random_sentence = "LMAO"

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)

prediction = model3.predict(padded_sequence)
predicted_index = np.argmax(prediction, axis=1)[0]

predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)


# Model 4: LSTM with sentiment

The code here makes use of the following resources to approach sentiment analysis within NN: https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91

The following refernece was used for TextBlob: https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from textblob import TextBlob
import tensorflow as tf

df2 = pd.read_csv('/content/dev_cleaned3.csv')

def get_sentiment(text):
    text = str(text)
    return TextBlob(text).sentiment.polarity

df2['Sentiment'] = df2['text'].apply(get_sentiment)

df2['Sentiment'] = (df2['Sentiment'] - df2['Sentiment'].mean()) / df2['Sentiment'].std()

In [63]:
X = df2[['text', 'Sentiment']]
y = df2['emoji']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train['text'])
X_train_sequences = tokenizer.texts_to_sequences(X_train['text'])
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=10)

X_test_sequences = tokenizer.texts_to_sequences(X_test['text'])
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=10)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train_encoded = label_encoder.transform(y_train)
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded, num_classes=len(label_encoder.classes_))

y_test_encoded = label_encoder.transform(y_test)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded, num_classes=len(label_encoder.classes_))


In [10]:
X_train_sentiment = X_train['Sentiment'].values
X_test_sentiment = X_test['Sentiment'].values

X_train_sentiment = X_train_sentiment.reshape(-1, 1)
X_test_sentiment = X_test_sentiment.reshape(-1, 1)


In [11]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, concatenate
from tensorflow.keras.models import Model

text_input = Input(shape=(10,), dtype='int32', name='text_input')
sentiment_input = Input(shape=(1,), dtype='float32', name='sentiment_input')

text_features = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100)(text_input)
text_features = Bidirectional(LSTM(64, return_sequences=True))(text_features)
text_features = Dropout(0.5)(text_features)
text_features = LSTM(32)(text_features)
text_features = Dropout(0.5)(text_features)

sentiment_features = Dense(16, activation='relu')(sentiment_input)

concatenated_features = concatenate([text_features, sentiment_features])

output = Dense(len(label_encoder.classes_), activation='softmax')(concatenated_features)

model = Model(inputs=[text_input, sentiment_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([X_train_padded, X_train_sentiment], y_train_categorical, epochs=10, validation_split=0.2)

loss, accuracy = model.evaluate([X_test_padded, X_test_sentiment], y_test_categorical)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10
4204/4204 [==============================] - 62s 13ms/step - loss: 2.9012 - accuracy: 0.2847 - val_loss: 2.6937 - val_accuracy: 0.3129
Epoch 2/10
4204/4204 [==============================] - 43s 10ms/step - loss: 2.6097 - accuracy: 0.3302 - val_loss: 2.6147 - val_accuracy: 0.3286
Epoch 3/10
4204/4204 [==============================] - 43s 10ms/step - loss: 2.4139 - accuracy: 0.3676 - val_loss: 2.6209 - val_accuracy: 0.3385
Epoch 4/10
4204/4204 [==============================] - 43s 10ms/step - loss: 2.2212 - accuracy: 0.4100 - val_loss: 2.6797 - val_accuracy: 0.3448
Epoch 5/10
4204/4204 [==============================] - 57s 13ms/step - loss: 2.0480 - accuracy: 0.4518 - val_loss: 2.7463 - val_accuracy: 0.3526
Epoch 6/10
4204/4204 [==============================] - 43s 10ms/step - loss: 1.8931 - accuracy: 0.4910 - val_loss: 2.9014 - val_accuracy: 0.3498
Epoch 7/10
4204/4204 [==============================] - 43s 10ms/step - loss: 1.7666 - accuracy: 0.5256 - val_loss: 3.0298 -

## Testing Model 4

In [23]:
predictions = model.predict([X_test_padded, X_test_sentiment])
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test_categorical, axis=1)



1314/1314 [==============================] - 6s 5ms/step


In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(true_classes, predicted_classes, average='weighted')

accuracy = accuracy_score(true_classes, predicted_classes)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.3562
Precision: 0.3371
Recall: 0.3562
F1 Score: 0.3290


In [29]:
random_sentence = "yay"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)


prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 76ms/step
yay [':clapping_hands:']


In [30]:
random_sentence = "I am so happy"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)


prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 34ms/step
I am so happy [':loudly_crying_face:']


In [31]:
random_sentence = "Bro I am so mad"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)

prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 27ms/step
Bro I am so mad [':face_with_tears_of_joy:']


In [32]:
random_sentence = "I love you"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)

prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 32ms/step
I love you [':two_hearts:']


In [33]:
random_sentence = "LMAO"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)

prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 60ms/step
LMAO [':face_with_tears_of_joy:']


# Model 5: Baseline with sentiment

In [64]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model


X_train, X_test, y_train, y_test = train_test_split(df2[['text', 'Sentiment']], df2['emoji'], test_size=0.2, random_state=42)

valid_indices = X_train['text'].notna() & y_train.notna()
X_train = X_train[valid_indices]
y_train = y_train[valid_indices]

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train['text'])
X_train_sequences = tokenizer.texts_to_sequences(X_train['text'])
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=10)

X_train_sentiment = X_train['Sentiment'].values.reshape(-1, 1)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_labels = label_encoder.transform(y_train)
y_train_categorical = tf.keras.utils.to_categorical(encoded_labels, num_classes=len(np.unique(y_train)))


text_input = Input(shape=(10,), dtype='int32', name='text_input')
text_features = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8, input_length=10)(text_input)
text_features = Flatten()(text_features)

sentiment_input = Input(shape=(1,), dtype='float32', name='sentiment_input')
sentiment_features = Dense(8, activation='relu')(sentiment_input)

concatenated_features = concatenate([text_features, sentiment_features])

output = Dense(len(np.unique(y_train)), activation='softmax')(concatenated_features)

model = Model(inputs=[text_input, sentiment_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None, 10)]                 0         []                            
                                                                                                  
 embedding_3 (Embedding)     (None, 10, 8)                1755104   ['text_input[0][0]']          
                                                                                                  
 sentiment_input (InputLaye  [(None, 1)]                  0         []                            
 r)                                                                                               
                                                                                                  
 flatten_1 (Flatten)         (None, 80)                   0         ['embedding_3[0][0]']   

In [65]:
model.fit([X_train_padded, X_train_sentiment], y_train_categorical, epochs=10, validation_split=0.2)


Epoch 1/10
30213/30213 [==============================] - 138s 5ms/step - loss: 2.6084 - accuracy: 0.3432 - val_loss: 2.4604 - val_accuracy: 0.3759
Epoch 2/10
30213/30213 [==============================] - 125s 4ms/step - loss: 2.3795 - accuracy: 0.3951 - val_loss: 2.4140 - val_accuracy: 0.3917
Epoch 3/10
30213/30213 [==============================] - 130s 4ms/step - loss: 2.3056 - accuracy: 0.4135 - val_loss: 2.4043 - val_accuracy: 0.3972
Epoch 4/10
30213/30213 [==============================] - 125s 4ms/step - loss: 2.2597 - accuracy: 0.4256 - val_loss: 2.4052 - val_accuracy: 0.4001
Epoch 5/10
30213/30213 [==============================] - 123s 4ms/step - loss: 2.2264 - accuracy: 0.4343 - val_loss: 2.4105 - val_accuracy: 0.4009
Epoch 6/10
30213/30213 [==============================] - 127s 4ms/step - loss: 2.2002 - accuracy: 0.4417 - val_loss: 2.4165 - val_accuracy: 0.4019
Epoch 7/10
30213/30213 [==============================] - 124s 4ms/step - loss: 2.1788 - accuracy: 0.4476 - val_

## Testing model 5

In [74]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

min_length = min(len(true_classes), len(predicted_classes))
true_classes = true_classes[:min_length]
predicted_classes = predicted_classes[:min_length]


precision, recall, f1, _ = precision_recall_fscore_support(true_classes, predicted_classes, average='weighted')
accuracy = accuracy_score(true_classes, predicted_classes)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.1943
Precision: 0.1294
Recall: 0.1943
F1 Score: 0.1472


In [75]:
random_sentence = "yay"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)


prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 19ms/step
yay [':smiling_face_with_smiling_eyes:']


In [76]:
random_sentence = "I am so happy"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)

prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 35ms/step
I am so happy [':loudly_crying_face:']


In [77]:
random_sentence = "Bro I am so mad"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)

prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 31ms/step
Bro I am so mad [':face_with_tears_of_joy:']


In [78]:
random_sentence = "I love you"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)

prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 80ms/step
I love you [':red_heart:']


In [79]:
random_sentence = "LMAO"
sentiment_value = get_sentiment(random_sentence)

sequence = tokenizer.texts_to_sequences([random_sentence])
padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
sentiment_input = np.array([sentiment_value]).reshape(1, -1)


prediction = model.predict([padded_sequence, sentiment_input])
predicted_index = np.argmax(prediction, axis=1)[0]
predicted_label = label_encoder.inverse_transform([predicted_index])
print(random_sentence, predicted_label)

1/1 [==============================] - 0s 68ms/step
LMAO [':face_with_tears_of_joy:']
